### 한국거래소(krx)에서 종목코드 가져오기

네이버금융에서 원하는 종목의 주식데이터를 가져오기 위해 먼저 코스피(KOSPI)과 코스닥(KOSDAQ)의 종목코드가 필요하다. 한국거래소(krx)에서는 주식시장에 상장된 기업들에 대해 종목코드를 제공한다. `pandas`모듈의 `pandas.read_html()`을 이용해 종목코드를 가져올 수 있다. pandas.read_html()은 HTML에서 `<table></table>`태그를 찾아 자동으로 `DataFrame` 형식으로 만들어준다. 아래의 소스코드를 통해 주식의 종목코드를 가져올 수 있다.

In [1]:
import pandas as pd 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 

# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] 

# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 
code_df.head()

,name,code
0,GS글로벌,001250
1,HSD엔진,082740
2,KG케미칼,001390
3,LG이노텍,011070
4,OCI,010060


In [9]:
code_df[:30]

,name,code
0,GS글로벌,001250
1,HSD엔진,082740
2,KG케미칼,001390
3,LG이노텍,011070
4,OCI,010060
5,SK네트웍스,001740
6,SK이노베이션,096770
7,STX,011810
8,WISCOM,024070
9,갤럭시아에스엠,011420


### 네이버금융(Naver금융)에서 종목 별 일자 데이터 가져오기

In [3]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 # 네이버 금융(http://finance.naver.com)에 넣어줌 
def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
    
    print("요청 URL = {}".format(url)) 
    return url 

# 신라젠의 일자데이터 url 가져오기 
item_name='신라젠' 
url = get_url(item_name, code_df) # 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() # 1페이지에서 20페이지의 데이터만 가져오기 

for page in range(1, 21): 
    pg_url = '{url}&page={page}'.format(url=url, page=page) 
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 
    
# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() 

# 상위 5개 데이터 확인하기 
df.head()

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=215600


,날짜,종가,전일비,시가,고가,저가,거래량
1,2019.05.17,56000.0,100.0,56700.0,57500.0,55900.0,385992.0
2,2019.05.16,55900.0,100.0,56000.0,57800.0,55400.0,558762.0
3,2019.05.15,56000.0,1200.0,55300.0,56600.0,54900.0,541148.0
4,2019.05.14,54800.0,1100.0,54300.0,56200.0,53600.0,995092.0
5,2019.05.13,55900.0,1500.0,56400.0,57700.0,55900.0,534330.0


In [6]:
# 한글로 된 컬럼명을 영어로 바꿔줌 

df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', 
                         '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 

# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)

# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 

# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 

# 상위 5개 데이터 확인 
df.head()

,date,close,diff,open,high,low,volume
298,2018-07-23,53600,8200,61300,62000,53300,2667093
297,2018-07-24,50300,3300,53800,54700,50300,3424127
296,2018-07-25,46300,4000,50300,50800,45300,7419416
295,2018-07-26,48900,2600,47300,50200,45600,5990446
294,2018-07-27,49400,500,49200,51800,48100,4038471


### Plotly를 이용해 Time Series 그래프 그리기

In [11]:
# 필요한 모듈 import 하기 
import plotly.offline as offline 
import plotly.graph_objs as go 

# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True) 

trace = go.Scatter( 
    x=df.date, 
    y=df.close, 
    name=item_name) 

data = [trace] 

# data = [celltrion] 
layout = dict( 
    title='{}의 종가(close) Time Series'.format(item_name), 
    xaxis=dict( 
        rangeselector=dict( 
            buttons=list([ 
                dict(count=1, label='1m', step='month', stepmode='backward'), 
                dict(count=3, label='3m', step='month', stepmode='backward'), 
                dict(count=6, label='6m', step='month', stepmode='backward'), 
                dict(step='all')
            ]) 
        ), 
        rangeslider=dict(), 
        type='date' 
    ) 
) 

fig = go.Figure(data=data, layout=layout) 
offline.iplot(fig)